# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use variables defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 2 from HW4 (theory part). 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.nodes[state]['demand'] = 1
G.nodes['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edges[s1, s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    try:
        # Create a copy of the original graph to add source and sink node
        transformed_graph = graph.copy()

        # Add a source and sink to the transformed graph and make demand be 0
        source = "SOURCE"
        sink = "SINK"
        transformed_graph.add_node(source, demand = 0)
        transformed_graph.add_node(sink, demand = 0)
        
        # Initialize total demand variable to make sure flow is feasible
        total_demand = 0 

        # Connect source to nodes with negative demand
        for node in graph.nodes():
            demand = graph.nodes[node].get('demand', 0)
            if demand < 0:
                transformed_graph.add_edge(source, node, capacity = - demand)

        # Connect nodes with positive demand to sink
        for node in graph.nodes():
            demand = graph.nodes[node].get('demand', 0)
            if demand > 0:
                transformed_graph.add_edge(node, sink, capacity = demand)
                # Add demand to total demand to get how much flow out of all graph
                total_demand = total_demand + demand

        # Run the maximum flow algorithm to find flow with demands
        total_flow, flow = nx.maximum_flow(transformed_graph, source, sink)
        
        # An error is thrown if the total flow is not equal to the total demand on the network
        if total_flow != total_demand:
            raise nx.NetworkXUnfeasible('No flow satisying the demands')
        
        # Remove source, sink, and all edges to and from source and sink from the flow 
        delete = ['SOURCE', 'SINK']
        for node in flow.values():
            for key in delete:
                if key in node:
                    del node[key]      
        del flow['SOURCE']
        del flow['SINK']

    
        return flow

    except nx.NetworkXUnfeasible:
        # An error is thrown if there is no flow satisfying the demands
        raise nx.NetworkXUnfeasible("No feasible flow satisfying the demands.")

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [5]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # Initialize a dictionary to store the total flow for each node and keys
    sum_flow = {}

    # Calculate outflow from each node
    for node, keys in flow.items():
        # Initialize the flow for the current node
        keys_flow = sum(curr_flow for curr_flow in keys.values())
        # Store the negative of the total outflow for the current node
        sum_flow[node] = - keys_flow

    # Calculate inflow to each state
    for keys in flow.values():
        for curr_key, curr_flow in keys.items():
            # Update the total flow for the current key, adding the inflow
            # If the key is not already in total_flow, use 0 as the default value
            sum_flow[curr_key] = curr_flow + sum_flow.get(curr_key, 0)
            
    # Return the final sum flow dictionary
    return sum_flow

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [6]:
flow = flow_with_demands(G)
div = divergence(flow)
#print ("Flow satisfies all demands:", all(div[n] == G.nodes[n]['demand'] for n in G.nodes()))